<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller: Identificando  Burger Master con MMG

El Burger Master es un evento creado en el 2016 por el *influencer* Tulio Zuluaga, más conocido en redes como Tulio recomienda, el cual busca que por una semana las hamburgueserías de cada ciudad ofrezcan su mejor producto a un precio reducido. 

El evento ha venido creciendo y en el 2022 se extendió por 21 ciudades de Colombia para las cuales se estimó que se vendieron más de dos millones de hamburguesas. El objetivo del presente caso-taller  es identificar los puntos calientes de hamburgueserías  que compitieron en  la ciudad de Bogotá aplicando el Modelo de Mezclas Gaussianas.

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo

### 1. Carga de datos  

En la carpeta `data` se encuentra el archivo `burger_master.xlsx` para la ciudad de Bogotá, cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from sklearn.mixture import GaussianMixture
import numpy as np
import matplotlib.pyplot as plt

: 

In [ ]:
df=pd.read_excel('burger_master.xlsx')
df.head()
df.shape
df['Restaurante'].nunique()

: 

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

### 2.  Visualizando los datos

Visualice la ubicación de cada restaurante en un mapa interactivo. Añada un marcador para cada restaurante y la posibilidad de encontrar la descripción de la hamburguesa ofrecida en un pop-up. (Note que la columna Descripción contiene otra información adicional).

In [ ]:
hamburguesas=gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Longitud, df.Latitud))

: 

In [ ]:
map = folium.Map(location = [hamburguesas.Latitud.mean(), hamburguesas.Longitud.mean()], tiles = "OpenStreetMap", zoom_start = 12)

for i in range(0,len(hamburguesas)):
   folium.Marker(
      location=[hamburguesas.iloc[i]['Latitud'], hamburguesas.iloc[i]['Longitud']]
   ).add_to(map)

for _, r in hamburguesas.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': '#FFFFFF'})
    folium.Popup(r['Descripción']).add_to(geo_j)
    geo_j.add_to(map)
map

: 

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

### 3.  Análisis de puntos calientes

Aplique el modelo de Mezclas Gaussianas para buscar clusters de restaurantes en Bogotá, mencione qué estructura de covarianza usó y explique por qué. Escoja el número óptimo de componentes, explicando el procedimiento y justificando su elección.

In [ ]:
X = hamburguesas[['Longitud','Latitud']].values

: 

In [ ]:
n_components = np.arange(2, 40)
models = [GaussianMixture(n, covariance_type='full', init_params= 'kmeans', random_state=123).fit(X)
          for n in n_components]

plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('Número de componentes')

: 

In [ ]:
n_components = np.arange(2, 40)
models = [GaussianMixture(n, covariance_type='full', init_params= 'kmeans', random_state=123).fit(X)
          for n in n_components]

plt.plot(n_components, [-m.score(X) for m in models], label='Log-Verosimilitud (neg)')
plt.xlabel('Número de componentes')

: 

In [ ]:
MMG_hamburguesas = GaussianMixture(n_components=3, covariance_type='full', init_params= 'kmeans', random_state=123)
labels = MMG_hamburguesas.fit(X).predict(X)

: 

In [ ]:
hamburguesas['cluster']=labels

: 

In [ ]:
map = folium.Map(location = [hamburguesas.Latitud.mean(), hamburguesas.Longitud.mean()], tiles = "OpenStreetMap", zoom_start = 12)

colour = ['red',
    'orange',
    'green',
    'purple',
    'cadetblue',
    'pink']

def colorCall(val):
  val = int(val)
  return colour[val]

#capa clusters de teatros
for Latitud, Longitud, c in zip(hamburguesas['Latitud'], hamburguesas['Longitud'], hamburguesas['cluster']):
    folium.CircleMarker(
        [Latitud, Longitud],
        radius=8,
        fill_color = colorCall(c),
        fill=True,
        color=colorCall(c),
        fill_opacity=0.7
        ).add_to(map) 

map

: 

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

#### 3.1. Visualización de los resultados

Visualice las densidades estimadas por el  mejor modelo estimado en la sección anterior usando un mapa de calor interactivo, discuta los resultados.

In [8]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

### 4. Comparación con KDE

Estime ahora las densidades usando KDE bivariado de la librería `statsmodels` con el anchos de banda dado por `cv_ml`. Muestre los resultados usando un mapa interactivo. Compare los resultados obtenidos por el "mejor" modelo encontrado via MMG. 

In [9]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)